In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

d:\Python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the data from Chavez, et al. (2017)
chavez_dt = pd.read_csv('data/chavez_data.csv')

# questionnaire design
pregunta = np.array([54,0,55,117,55,0,75,61,19,0,25,53,31,0,85,7,
                     14,0,25,19,47,0,50,160,15,0,35,13,25,0,60,14,
                     78,0,80,162,40,0,55,62,11,0,30,7,67,0,75,119,
                     34,0,35,186,27,0,50,21,69,0,85,91,49,0,60,89,
                     80,0,85,157,24,0,35,29,33,0,80,14,28,0,30,179,
                     34,0,50,30,25,0,30,80,41,0,75,20,54,0,60,111,
                     54,0,80,30,22,0,25,136,20,0,55,7]).reshape(27, 4)

condition_col = np.array(['DT'+str(i) for i in range(1, 28)]).reshape(-1,1)
pregunta = np.hstack((condition_col,pregunta))
pregunta = pd.DataFrame(pregunta)
condition_col_name = ['ss_x','ss_t','ll_x','ll_t']
pregunta.columns = ['condition'] + condition_col_name

# object containing participants' choices: '0' is choice of SS, '1' is choice of 'LL'
chavez_dt = pd.melt(chavez_dt, id_vars=['ID','School'], var_name='condition', value_name='choice')
chavez_dt = pd.merge(chavez_dt,pregunta,on='condition')
chavez_dt[condition_col_name] = chavez_dt[condition_col_name].apply(pd.to_numeric)
chavez_dt.columns = ['person_id','school','condition','choice'] + condition_col_name

In [23]:
df_group = chavez_dt.groupby(['ss_x','ss_t','ll_x','ll_t'])['choice']
choice_group = df_group.mean('choice')
choice_correct = choice_group *(choice_group > 0.5) +  (1-choice_group) *(choice_group <= 0.5)

sum(choice_correct * df_group.count()) / sum(df_group.count())

0.7767637259162683

In [27]:
itch_dt = pd.read_csv('data/ericson_data.csv')
itch_dt = itch_dt.rename(columns={"Subject":"person_id",
                                "Condition":"condition",
                                "Question":"question_id",
                                "X1":"ss_x",
                                "T1":"ss_t",
                                "X2":"ll_x",
                                "T2":"ll_t",
                                "LaterOptionChosen": "choice"}).\
                drop(['R','G','D'],axis=1).dropna()

In [29]:
df_group = itch_dt.groupby(['ss_x','ss_t','ll_x','ll_t'])['choice']
choice_group = df_group.mean('choice')
choice_correct = choice_group *(choice_group > 0.5) +  (1-choice_group) *(choice_group <= 0.5)

sum(choice_correct * df_group.count()) / sum(df_group.count())

0.7120314729151356

In [19]:
# Convert training and testing data to PyTorch tensors
train_data = chavez_dt.dropna()
X = torch.tensor(train_data[['ss_x', 'll_x', 'ss_t', 'll_t']].values, dtype=torch.float32)
y = torch.tensor(train_data['choice'].values, dtype=torch.float32)

In [20]:
# Define the neural network
class NetV(nn.Module):
    def __init__(self):
        super(NetV, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class NetClassifier(nn.Module):
    def __init__(self):
        super(NetClassifier, self).__init__()
        self.v_s = NetV()
        self.v_l = NetV()
        self.logit_temperature = nn.Parameter(torch.tensor(1.0), requires_grad=True)

    def forward(self, ss_x, ss_t, ll_x, ll_t):
        v_s = self.v_s(torch.stack([ss_x, ss_t], dim=1))
        v_l = self.v_l(torch.stack([ll_x, ll_t], dim=1))
        diff = v_l - v_s
        logit_input = diff * self.logit_temperature
        prob = torch.sigmoid(logit_input)
        return prob

In [21]:
from torch.utils.data import DataLoader, TensorDataset

# Initialize the classifier
classifier = NetClassifier()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Define the number of epochs and batch size
epochs = 50
batch_size = 32

# Split the data into training and validation sets
dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Training loop
for epoch in range(epochs):
    classifier.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        prob = classifier(*batch_x.T)
        loss = criterion(prob.view(-1), batch_y)
        loss.backward()
        optimizer.step()

    # Validation
    classifier.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            prob = classifier(*batch_x.T)
            val_loss += criterion(prob.view(-1), batch_y).item()
            predicted = (prob >= 0.5).float()
            correct += predicted.eq(batch_y.view_as(predicted)).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100. * correct / len(val_dataset)
    print(f'Epoch {epoch + 1}/{epochs}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

Epoch 1/50, Val Loss: 0.5250, Accuracy: 76.85%
Epoch 2/50, Val Loss: 0.5072, Accuracy: 77.01%
Epoch 3/50, Val Loss: 0.5013, Accuracy: 77.01%
Epoch 4/50, Val Loss: 0.5186, Accuracy: 76.85%
Epoch 5/50, Val Loss: 0.5040, Accuracy: 77.01%
Epoch 6/50, Val Loss: 0.5063, Accuracy: 77.01%
Epoch 7/50, Val Loss: 0.5080, Accuracy: 75.82%
Epoch 8/50, Val Loss: 0.5025, Accuracy: 77.01%
Epoch 9/50, Val Loss: 0.5003, Accuracy: 77.01%
Epoch 10/50, Val Loss: 0.5023, Accuracy: 77.01%
Epoch 11/50, Val Loss: 0.5017, Accuracy: 77.01%
Epoch 12/50, Val Loss: 0.4996, Accuracy: 77.01%
Epoch 13/50, Val Loss: 0.4987, Accuracy: 77.01%
Epoch 14/50, Val Loss: 0.5049, Accuracy: 75.98%
Epoch 15/50, Val Loss: 0.4993, Accuracy: 77.01%
Epoch 16/50, Val Loss: 0.5017, Accuracy: 77.01%
Epoch 17/50, Val Loss: 0.5030, Accuracy: 77.01%
Epoch 18/50, Val Loss: 0.5007, Accuracy: 77.01%
Epoch 19/50, Val Loss: 0.4986, Accuracy: 77.01%
Epoch 20/50, Val Loss: 0.4982, Accuracy: 76.85%
Epoch 21/50, Val Loss: 0.4995, Accuracy: 77.01%
E